In [1]:
import pandas as pd
import geopandas as gpd
from unidecode import unidecode

# Estaciones del sistema de Trenes Argentinos

In [2]:
estaciones = pd.read_csv("https://servicios.transporte.gob.ar/gobierno_abierto/descargar.php?t=estaciones&d=estaciones", encoding="UTF-8", sep=";")
estaciones

,mes,linea,ramal,estacion,latitud,longitud
0,12/2018,Urquiza,Lemos,Tropezón,-34.591953,-58.559179
1,12/2018,Urquiza,Lemos,Teniente Agneta,-34.548800,-58.678850
2,12/2018,Urquiza,Lemos,Sargento Cabral,-34.537187,-58.694932
3,12/2018,Urquiza,Lemos,Sargento Barrufaldi,-34.562450,-58.660275
4,12/2018,Urquiza,Lemos,Rubén Darío,-34.587484,-58.631454
...,...,...,...,...,...,...
3017,06/2017,Belgrano Norte,Villa Rosa,Don Torcuato,-34.502657,-58.642002
3018,06/2017,Belgrano Norte,Villa Rosa,Del Viso,-34.453694,-58.796119
3019,06/2017,Belgrano Norte,Villa Rosa,Carapachay,-34.528805,-58.537264
3020,06/2017,Belgrano Norte,Villa Rosa,Boulogne Sur,-34.508711,-58.565298


In [3]:
#DEJAMOS TODO EN MAYUS - NOMBRES DE COLUMNAS
estaciones.columns = estaciones.columns.str.upper()

In [4]:
#QUITAMOS TILDES Y DEJAMOS TODO EN MAYUS - VALORES
estaciones[["LINEA", "RAMAL", "ESTACION"]]= estaciones[["LINEA", "RAMAL", "ESTACION"]].applymap(lambda x: unidecode(x).upper())

In [5]:
#DAMOS FORMATO A LA FECHA
estaciones["FECHA"] = pd.to_datetime(estaciones["MES"]).dt.strftime("%m-%Y")

In [6]:
estaciones

,MES,LINEA,RAMAL,ESTACION,LATITUD,LONGITUD,FECHA
0,12/2018,URQUIZA,LEMOS,TROPEZON,-34.591953,-58.559179,12-2018
1,12/2018,URQUIZA,LEMOS,TENIENTE AGNETA,-34.548800,-58.678850,12-2018
2,12/2018,URQUIZA,LEMOS,SARGENTO CABRAL,-34.537187,-58.694932,12-2018
3,12/2018,URQUIZA,LEMOS,SARGENTO BARRUFALDI,-34.562450,-58.660275,12-2018
4,12/2018,URQUIZA,LEMOS,RUBEN DARIO,-34.587484,-58.631454,12-2018
...,...,...,...,...,...,...,...
3017,06/2017,BELGRANO NORTE,VILLA ROSA,DON TORCUATO,-34.502657,-58.642002,06-2017
3018,06/2017,BELGRANO NORTE,VILLA ROSA,DEL VISO,-34.453694,-58.796119,06-2017
3019,06/2017,BELGRANO NORTE,VILLA ROSA,CARAPACHAY,-34.528805,-58.537264,06-2017
3020,06/2017,BELGRANO NORTE,VILLA ROSA,BOULOGNE SUR,-34.508711,-58.565298,06-2017


In [13]:
#RED DE TREN GEOREFERENCIADA
filtro = ["BELGRANO SUR", "MITRE", "ROCA" "SAN MARTIN", "SARMIENTO"] 
estaciones = estaciones[estaciones["LINEA"].isin(filtro)].reset_index(drop=True)

#### SELECCIONAMOS LAS ESTACIONES MAS RECIENTES

In [14]:
#ordenamos por fecha
estaciones = estaciones.sort_values(by="FECHA", ascending=False).reset_index(drop=True)

In [15]:
#eliminamos los duplicados pero dejamos el indice
estaciones_unicas = estaciones[["LINEA", "RAMAL", "ESTACION"]].drop_duplicates()

In [16]:
estaciones_merge = pd.merge(estaciones_unicas, estaciones[["LATITUD", "LONGITUD", "FECHA"]], left_index=True, right_index=True, how="left")

In [17]:
estaciones_merge = estaciones_merge.sort_values(by=["FECHA", "LINEA", "RAMAL", "ESTACION"]).reset_index(drop=True)

In [18]:
estaciones_merge

,LINEA,RAMAL,ESTACION,LATITUD,LONGITUD,FECHA
0,BELGRANO SUR,GONZALEZ CATAN,BUENOS AIRES,-34.645562,-58.393995,06-2017
1,BELGRANO SUR,M.C.G. BELGRANO,BUENOS AIRES,-34.645562,-58.393995,06-2017
2,MITRE,TIGRE,LISANDRO DE LA TORRE,-34.562529,-58.436711,06-2017
3,MITRE,TIGRE,L. DE LA TORRE,-34.562529,-58.436711,10-2018
4,BELGRANO SUR,APEADERO KM 12,ING. BUDGE,-34.718663,-58.462151,11-2018
...,...,...,...,...,...,...
158,SARMIENTO,MORENO,ONCE,-34.608591,-58.407559,12-2018
159,SARMIENTO,MORENO,PASO DEL REY,-34.658266,-58.761941,12-2018
160,SARMIENTO,MORENO,RAMOS MEJIA,-34.640384,-58.563947,12-2018
161,SARMIENTO,MORENO,S.A. DE PADUA,-34.665425,-58.697997,12-2018


## EXPORT

In [20]:
estaciones_merge.to_csv("./data/05-TRENES-ESTACIONES.csv", index=False)

In [21]:
import geopandas as gpd

In [28]:
geo_estaciones_merge = gpd.GeoDataFrame(estaciones_merge, geometry=gpd.points_from_xy(estaciones_merge["LONGITUD"], estaciones_merge["LATITUD"]), crs={'init' : 'epsg:4326'})

In [31]:
geo_estaciones_merge.to_file("./maps/AR-FERRORARRIL-ESTACIONES.geojson", driver="GeoJSON", encodign="UTF-8")